In [1]:
import numpy as np
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Flatten, Conv1D, MaxPooling1D, BatchNormalization, Activation
from tensorflow.keras.activations import relu
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.metrics import Accuracy, AUC
from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [4]:
y = np.load('y.npy')
y = 1 * (y > 2)

X = np.load('X.npy')
X.shape

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = MinMaxScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

y_train = y_train.reshape(y_train.shape[0], 1)
y_test = y_test.reshape(y_test.shape[0], 1)

(4320, 20480)

In [3]:
# counts = np.bincount(y_test[:, 0])

# weight_for_0 = 1.0 / counts[0]
# weight_for_1 = 1.0 / counts[1]

In [35]:
X_train.shape, y_train.shape
np.sum(y_test > 0), np.sum(y_test <= 0)
# np.sum(y_train > 2), np.sum(y_train <= 2)
# 310/(310+266) = 53.8%

(445, 419)

In [5]:
es = EarlyStopping(monitor="val_auc", patience=20, baseline=0.5, mode='max', restore_best_weights=True)
class_weight = {0: weight_for_0, 1: weight_for_1}

model = Sequential()

model.add(Conv1D(filters=8, kernel_size=2, input_shape=X_train[0].shape))
# model.add(BatchNormalization())
model.add(Activation(relu))
model.add(MaxPooling1D(pool_size=4))

model.add(Conv1D(filters=16, kernel_size=2))
# model.add(BatchNormalization())
model.add(Activation(relu))
model.add(MaxPooling1D(pool_size=4))
          
model.add(Conv1D(filters=16, kernel_size=2))
# model.add(BatchNormalization())
model.add(Activation(relu))
model.add(MaxPooling1D(pool_size=8))

model.add(Conv1D(filters=8, kernel_size=2))
# model.add(BatchNormalization())
model.add(Activation(relu))
model.add(MaxPooling1D(pool_size=8))

model.add(Flatten())
model.add(Dense(50))
# model.add(BatchNormalization())
model.add(Activation(relu))

model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[AUC()])

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 20479, 8)          24        
_________________________________________________________________
activation (Activation)      (None, 20479, 8)          0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 5119, 8)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 5118, 16)          272       
_________________________________________________________________
activation_1 (Activation)    (None, 5118, 16)          0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 1279, 16)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 1278, 16)          5

In [8]:
# fit model
history = model.fit(X_train, y_train, epochs=1, validation_split=0.2, callbacks=[es])#, class_weight=class_weight)

87/87 [==============================] - 6s 70ms/step - loss: 0.3610 - auc: 0.9199 - val_loss: 0.6258 - val_auc: 0.7943


In [33]:
# from sklearn.metrics import roc_auc_score, plot_roc_curve, roc_curve
# import matplotlib.pyplot as plt
# import scikitplot as skplt

# # roc_auc = roc_auc_score(y_test, model.predict_proba(X_test))
# # skplt.metrics.plot_roc_curve(y_test, model.predict_proba(X_test))
# # plt.show()
# metrics.roc_curve(y_test, model.predict_proba(X_test))

In [34]:
# fpr, tpr, thresholds = roc_curve(y_test, model.predict_proba(X_test), pos_label=2)
# plt.plot(fpr, tpr, 'g', alpha=0.15)
# plt.xlim([-0.01, 1.01])
# plt.ylim([-0.01, 1.01])